# Agentic AI Workshop

# Introducing: The Price Is Right

We are going to build a multi-agent framework that will:
1. Pull RSS feeds from the web and interpret them, looking for promising deals (multi-shot prompting, structured outputs)
2. Use RAG and a Frontier Model to estimate the price of a product, using a knowledge base
3. Use a QLoRA fine tuned LLM to estimate the price
4. Send a push notification when it finds a deal that's worth significantly less than it estimates
5. A planning agent will orchestrate this

If you're having any set up problems, please see the [troubleshooting](../troubleshooting.ipynb) notebook in the parent directory.  
For more details on using Jupyter Lab, please see the [Guide To Jupyter](../Guide%20to%20Jupyter.ipynb) notebook.  
If you'd like to brush up your Python skills, there's an [Intermediate Python](../Intermediate%20Python.ipynb) notebook.


## First: Let's look at the Agent code for interpreting RSS feeds, including figuring out the pricing, using multi-shot prompting

## Remember: just get some intuition for this, and run it yourself later..

In [3]:
# Some imports
import os

from dotenv import load_dotenv
from price_agents.deals import ScrapedDeal, DealSelection
from openai import OpenAI, AsyncOpenAI
import logging

from agents import Agent, Runner, set_default_openai_client
from agents.run import DEFAULT_AGENT_RUNNER

In [5]:
# Set up our env variables

load_dotenv()
openai = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"), base_url="https://openrouter.ai/api/v1"
)
MODEL = "openai/gpt-5-mini"

In [6]:
# I wrote this code to fetch RSS feeds

scraped_deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 3/3 [00:43<00:00, 14.35s/it]


In [7]:
print(scraped_deals[2].describe())

Title: Samsung Phone Deals: Up to $1,000 off w/ trade, storage upgrades, more + free shipping
Details: Samsung is offering up to $1,000 off select Galaxy phones with eligible trade-ins, along with additional perks like free storage upgrades and bonus credits on certain models. It's a strong opportunity to save big on the latest devices while getting extra value from your old phone. Shop Now at Samsung
Features: 
URL: https://www.dealnews.com/Samsung-Phone-Deals-Up-to-1-000-off-w-trade-storage-upgrades-more-free-shipping/21799129.html?iref=rss-c142


## We are going to ask GPT-5-mini to summarize deals and identify their price

In [8]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [9]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [10]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(scraped_deals)
print(user_prompt[:2000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung Watches & Wearables Deals: Up to $1,140 off
Details: Save on smart watches, smart rings, earbuds, and more. Not only will you find discounts, but also trade-in credits on qualifying devices. We've pictured the Samsung Galaxy Watch8 46mm Classic Smartwatch for $450 ($50 off or up to $250 off with trade-in). Shop Now at Samsung
Features: 
URL: https://www.dealnews.com/Sam

## Calling OpenAI with Structured Outputs

In [11]:
response = openai.chat.completions.parse(model=MODEL, messages=messages, response_format=DealSelection, reasoning_effort="minimal")
results = response.choices[0].message.parsed
results

DealSelection(deals=[Deal(product_description='Movespeed S10Pro is a compact 10,000 mAh magnetic power bank designed for portable charging. It includes magnetic alignment for compatible phones, a slim housing for easy carry, and enough capacity to recharge most smartphones one to two times depending on battery size. It ships from a U.S. warehouse and is suited for everyday carry and travel charging needs.', price=22.0, url='https://www.dealnews.com/Movespeed-S10-Pro-10-000-m-Ah-Magnetic-Power-Bank-for-22-free-shipping/21799066.html?iref=rss-c142'), Deal(product_description="Unlocked Apple iPhone 17 Air with 256 GB of storage is a slim 5G-capable smartphone offering Apple's latest chipset and camera system in a lighter Air form factor. The 256 GB model provides ample onboard storage for apps, photos, and media without relying on cloud storage. It comes unlocked for use on compatible carriers and includes standard accessories and free shipping from the seller.", price=969.0, url='https:/

In [12]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.url)
    print(deal.price)
    print()

Movespeed S10Pro is a compact 10,000 mAh magnetic power bank designed for portable charging. It includes magnetic alignment for compatible phones, a slim housing for easy carry, and enough capacity to recharge most smartphones one to two times depending on battery size. It ships from a U.S. warehouse and is suited for everyday carry and travel charging needs.
https://www.dealnews.com/Movespeed-S10-Pro-10-000-m-Ah-Magnetic-Power-Bank-for-22-free-shipping/21799066.html?iref=rss-c142
22.0

Unlocked Apple iPhone 17 Air with 256 GB of storage is a slim 5G-capable smartphone offering Apple's latest chipset and camera system in a lighter Air form factor. The 256 GB model provides ample onboard storage for apps, photos, and media without relying on cloud storage. It comes unlocked for use on compatible carriers and includes standard accessories and free shipping from the seller.
https://www.dealnews.com/Unlocked-Apple-iPhone-17-Air-256-GB-5-G-Smart-Phone-for-969-free-shipping/21799027.html?ire

# Putting this into an "Agent"

I've packaged this code into a class called `ScannerAgent`

There are various Agent frameworks that add an abstraction layer, but in our case it's easy just to write the code directly.

In [13]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [14]:
from price_agents.scanner_agent import ScannerAgent

scanner = ScannerAgent()
scanner.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 15 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


DealSelection(deals=[Deal(product_description='Movespeed S10Pro is a portable 10,000 mAh magnetic power bank designed for on-the-go charging. It features a magnetic attachment for compatible phones, a compact rectangular build for pocketable carry, and delivers multiple full charges for most smartphones. The unit ships from a U.S. warehouse and is intended as an affordable backup battery for daily use and travel.', price=22.0, url='https://www.dealnews.com/Movespeed-S10-Pro-10-000-m-Ah-Magnetic-Power-Bank-for-22-free-shipping/21799066.html?iref=rss-c142'), Deal(product_description='Unlocked Apple iPhone 17 Air with 256 GB storage is a slim 5G-capable smartphone in Apple’s Air lineup featuring a high-resolution display, modern mobile processor, and a large internal storage tier suitable for heavy photo, video, and app usage. It’s sold unlocked for use with multiple carriers and includes standard smartphone features like advanced cameras, wireless connectivity, and iOS software.', price=